In [2]:
import requests
import pandas as pd
import time
from datetime import datetime

# Set API key and base URL (Replace with actual API key if needed)
EXTRACTOR_API_KEY = "4f91af1484d341a2683adc1106faf6bea22442c1e715729f4786a32fb551fd2c"
EXTRACTOR_API_URL = "https://api.sec-extractor.com/extract"

# Define headers
HEADERS = {
    "User-Agent": "sude.tiras@sabanciuniv.edu"
}

# Function to get filings from EDGAR Extractor API
def get_filing_data(cik, filing_type="10-Q", start_year=2000, end_year=datetime.today().year):
    filings = []
    for year in range(start_year, end_year + 1):
        url = f"https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={cik}&type={filing_type}&dateb=&owner=exclude&start=0&count=100"
        response = requests.get(url, headers=HEADERS)
        if response.status_code == 200:
            try:
                # Extract filing links
                filings.append(response.json())
            except ValueError:
                print(f"Error decoding JSON for URL: {url}")
                continue
        time.sleep(1)  # Rate limiting to avoid being blocked
    return filings

# Function to extract MD&A and Risk Factors sections
def extract_sections(filing_url):
    response = requests.post(
    if response.status_code == 200:
        try:
            data = response.json()
            return data.get("Management’s Discussion and Analysis", ""), data.get("Risk Factors", "")
        except ValueError:
            print(f"Error decoding JSON for URL: {filing_url}")
            return "", ""
    )
    if response.status_code == 200:
        data = response.json()
        return data.get("Management’s Discussion and Analysis", ""), data.get("Risk Factors", "")
    return "", ""

# List of companies (CIKs, tickers, and names)
companies = [
    {"cik": "0000320193", "ticker": "AAPL", "name": "Apple Inc."},
    {"cik": "0000789019", "ticker": "MSFT", "name": "Microsoft Corp"},
    # Add more companies as needed
]

# Collect data
data = []
for company in companies:
    cik = company["cik"]
    ticker = company["ticker"]
    name = company["name"]

    filings = get_filing_data(cik)
    for filing in filings:
        for item in filing.get("items", []):
            filing_url = item["filingUrl"]
            filing_date = item["filingDate"]
            quarter = f"Q{((int(filing_date.split('-')[1]) - 1) // 3) + 1}"

            # Extract MD&A and Risk Factors
            mdna, risk_factors = extract_sections(filing_url)
            data.append([cik, ticker, name, filing_date, quarter, mdna, risk_factors])

            time.sleep(1)  # Avoid hitting API rate limits

# Save data to CSV
df = pd.DataFrame(data, columns=["CIK", "Ticker", "Company Name", "Date", "Quarter", "MD&A", "Risk Factors"])
df.to_csv("sec_10q_data.csv", index=False)

print("Data extraction complete. CSV file saved.")


SyntaxError: invalid syntax (1523383097.py, line 34)